## Exploration4. 멋진 작사가 만들기

### >데이터 불러오기

In [1]:
import glob
import os, re
import tensorflow as tf
import numpy as np 

txt_file_path = os.getenv('HOME')+'/aiffel/data_preprocess/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()  #텍스트를 라인 단위로 읽어오기
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof']


### > 데이터 다듬기

불러온 데이터에 괄호 등 불필요한 부분이 없으므로 공백인 문장 또는 중복문장 제외하기

In [2]:
corpus= []

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue           # 길이가 0인 문장 제외
    
    corpus.append(sentence)


print(f'원본 데이터 크기 : {len(raw_corpus)}')
print(f'공백인 문장을 제외한 데이터 크기 : {len(corpus)}')

corpus = list(set(corpus))          # 중복되는 문장 제외 후 다시 리스트로 변환
print(f'중복되는 문장 제외한 데이터 크기 : {len(corpus)}')

원래 데이터 크기 : 187088
공백인 문장, 대괄호 포함된 문장 제외한 데이터 크기 : 174446
중복되는 문장 제외한 데이터 크기 : 117114


### > 토큰화

1. 소문자로 바꾸고, 양쪽 공백 삭제
2. 특수문자 양쪽에 공백 삽입
3. 여러개의 공백은 하나의 공백으로 바꿈
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿈
5. 다시 양쪽 공백 삭제
6. 문장 시작에는 <start>, 끝에는 <end>를 추가

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                     # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)     # 2
    sentence = re.sub(r'[" "]+', " ", sentence)             # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)   # 4
    sentence = sentence.strip()                             # 5
    sentence = '<start> ' + sentence + ' <end>'             # 6
    return sentence

In [4]:
corpus_tokenized = []

for sentence in corpus:
    
    
    # 데이터 정제하기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus_tokenized.append(preprocessed_sentence)
        
# 10개의 결과만 확인
corpus_tokenized[:10]

['<start> but treat dimes fair and im <end>',
 '<start> and i d take up for him <end>',
 '<start> i tell you what , it don t bother me nohow , <end>',
 '<start> where everything you want <end>',
 '<start> you give it one more chance <end>',
 '<start> i ma be around forever , entertain even in the ground <end>',
 '<start> i like my ice frozen like the antartic <end>',
 '<start> id rather spend a ching aling on it eh eh eh <end>',
 '<start> but i know what you are <end>',
 '<start> straight up how many times i gotta tell that ass to come over ? <end>']

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,         # 12000개 단어를 기억할 수 있음
        filters=' ',
        oov_token="<unk>"        # 포함되지 않는 단어는 <unk> 으로 표현
    )
    
    tokenizer.fit_on_texts(corpus)
    

    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus_tokenized)

[[  2  31 850 ...   0   0   0]
 [  2   8   5 ...   0   0   0]
 [  2   5  91 ...   4   3   0]
 ...
 [  2   8   9 ... 370  54   3]
 [  2   8 399 ...   0   0   0]
 [  2  22  79 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f7f7b830a00>


### > 단어사전 보기

In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
src_input = tensor[:, :-1] # 소스문장 생성 

tgt_input = tensor[:, 1:]  # 타켓 문장 생성

print(src_input[0])
print(tgt_input[0])

[   2   31  850 5169 1169    8  120    3    0    0    0    0    0    0]
[  31  850 5169 1169    8  120    3    0    0    0    0    0    0    0]


### > 데이터셋 분리하기

In [8]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34) 

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (93691, 14)
Target Train: (93691, 14)


### > 모델학습

In [10]:
from tensorflow.keras.layers import Embedding, LSTM, Dense


class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = Embedding(vocab_size, embedding_size)
        self.rnn_1 = LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = LSTM(hidden_size, return_sequences=True)
        self.linear = Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

단어 벡터 차원의 수인 embedding_size, LSTM레이어의 hidden_size 조절해주기

In [11]:
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10) #epochs 값 10으로 지정

Epoch 1/10
2928/2928 [==============================] - 307s 97ms/step - loss: 3.4321
Epoch 2/10
2928/2928 [==============================] - 304s 104ms/step - loss: 2.9925
Epoch 3/10
2928/2928 [==============================] - 311s 106ms/step - loss: 2.6803
Epoch 4/10
2928/2928 [==============================] - 313s 107ms/step - loss: 2.3282
Epoch 5/10
2928/2928 [==============================] - 311s 106ms/step - loss: 1.9747
Epoch 6/10
2928/2928 [==============================] - 311s 106ms/step - loss: 1.6656
Epoch 7/10
2928/2928 [==============================] - 310s 106ms/step - loss: 1.4200
Epoch 8/10
2928/2928 [==============================] - 311s 106ms/step - loss: 1.2542
Epoch 9/10
2928/2928 [==============================] - 310s 106ms/step - loss: 1.1535
Epoch 10/10
2928/2928 [==============================] - 311s 106ms/step - loss: 1.0944


epoch 10으로 학습결과 loss는 1.0944이다

### > 모델평가하기

1. 입력받은 문장의 텐서를 입력함
2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냄
3. 2에서 예측된 word index를 문장 뒤에 붙인다.
4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성 종료

In [13]:
# generate_text 함수를 사용하여 작문 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
 
    while True:
        predict = model(test_tensor)  # 1
        
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        
        # 3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
     # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [14]:
generate_text(model, tokenizer, init_sentence="<start> you are", max_len=20)

'<start> you are the greatest thing to me . <end> '

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i always", max_len=20)

'<start> i always have respected her for busting out and gettin free <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> if", max_len=20)

'<start> if you re wondering , know that i m not sorry <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> there", max_len=20)

'<start> there s no one in life quite like you , you baby <end> '

#### > 회고

중간중간에 텐서플로우를 선언했는데 자꾸 오류가 떠서 고생했다.
결과물을 보면 생각보다 sweet한 문장을 만들어낸다. 모델이 감성적이다.
대충 이해해서 그렇지만 조금 어색한 문장이 있을것 같기도 하다.
노드 실습때는 할만하다고 생각했는데 몇일이 지나고 다시 보니
까먹어서 다시 훑어보았다.. 몇일 차이로 프로젝트가 조금더 어렵게 느껴졌다.
조금 더 다양한 전처리를 활용해보고 싶다.